# [Aligning Big Brains and Atlases](https://biop.github.io/ijp-imagetoatlas/) in Python

This notebook demos the use of ABBA with python. In particular it integrates [DeepSlice](https://github.com/PolarBean/DeepSlice) with ABBA.

Technically speaking, this notebook relies on [JPype](https://github.com/jpype-project/jpype) and [PyImageJ](https://github.com/imagej/pyimagej) to make the bridge between Python and Java. Surprisingly, there's no almost no functionality loss...

Note that ABBA works for aligning serial sections. If you want to register a nice 3D dataset, you'd rather have a look at [brainreg](https://github.com/brainglobe/brainreg).


## What happens in this notebook ?

We start ABBA, download a few serial sections example, register them with DeepSlice. Finally non-linear registration.


In [1]:
# Starting PyImageJ, some of these dependencies may be autodiscovered via transitive dependencies, not sure

imagej_core_dep = 'net.imagej:imagej:2.3.0'
imagej_legacy_dep = 'net.imagej:imagej-legacy:0.38.1'
abba_dep = 'ch.epfl.biop:ImageToAtlasRegister:0.2.13'

deps_pack = [imagej_core_dep, imagej_legacy_dep, abba_dep]

In [2]:
# Starts ImageJ, show UI
import imagej
ij = imagej.init(deps_pack, mode='interactive')
ij.ui().showUI()

In [3]:
# Starts ABBA
from scyjava import jimport
import jpype
import jpype.imports
from jpype.types import *
from jpype import JImplements, JOverride

File = jimport('java.io.File')

# .. but before : logger, please shut up
DebugTools = jimport('loci.common.DebugTools')
DebugTools.enableLogging('INFO')

# Ok, let's start ABBA and its BDV view (it's also possible to start it without any GUI, 
# or even to build another GUI with a Napari view, why not ?)
ABBAOpenAtlasCommand = jimport('ch.epfl.biop.atlas.aligner.command.ABBAOpenAtlasCommand') # Command to open an atlas

ba = ij.command().run(ABBAOpenAtlasCommand, True, 'atlasType', 'Adult Mouse Brain - Allen Brain Atlas V3').get().getOutput("atlas")

ABBABdvStartCommand = jimport('ch.epfl.biop.atlas.aligner.gui.bdv.ABBABdvStartCommand') # Command import
ij.command().run(ABBABdvStartCommand, True, 'slicing_mode', 'coronal', 'ba', ba) # Starts it with the converted brainglobe atlas in the coronal orientation

[java.lang.Enum.toString] [WARNING] Ignoring unsupported output: ba [ch.epfl.biop.atlas.mouse.allen.ccfv3.command.AllenBrainAdultMouseAtlasCCF2017Command]
[java.lang.Enum.toString] [WARNING] Ignoring unsupported output: atlas [ch.epfl.biop.atlas.mouse.allen.ccfv3.command.AllenBrainAdultMouseAtlasCCF2017Command]


<java object 'java.util.concurrent.FutureTask'>

[java.lang.Enum.toString] Creating MultiSlicePositioner instance
[java.lang.Enum.toString] MultiSlicePositioner instance created
[java.lang.Enum.toString] [WARNING] Ignoring unsupported output: mp [ch.epfl.biop.atlas.aligner.MultiSlicePositioner]
[java.lang.Enum.toString] Registration plugin RegistrationElastixAffineCommand discovered
[java.lang.Enum.toString] Registration plugin RegistrationElastixAffineRemoteCommand discovered
[java.lang.Enum.toString] Registration plugin RegistrationBigWarpCommand discovered
[java.lang.Enum.toString] Registration plugin RegistrationElastixSplineCommand discovered
[java.lang.Enum.toString] Registration plugin RegistrationElastixSplineRemoteCommand discovered
[java.lang.Enum.toString] [WARNING] Ignoring unsupported output: view [ch.epfl.biop.atlas.aligner.gui.bdv.BdvMultislicePositionerView]


In [4]:
# Import deepslice and make the function for the ABBA command
Function = jimport('java.util.function.Function')


In [5]:
from DeepSlice import DeepSlice

@JImplements(Function)
class DeepSliceProcessor:
    @JOverride
    def apply(self,folder):
        Model = DeepSlice()
        Model.Build()
        print(folder.getParent())
        Model.predict('deepslice')#str(folder.getParent()))
        out = File(folder,JString('results'))
        print(out.getAbsolutePath())
        Model.Save_Results(out.getAbsolutePath())
        return File(folder,JString('results.xml'))

C:\Users\chiarutt\Dropbox\BIOP\ABBA-Python\notebooks


In [6]:
run_deep_slice = DeepSliceProcessor()


In [7]:
run_deep_slice.apply(File('deepslice/images'))

set one thread
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
deepslice
define_image_generator
Found 9 images belonging to 1 classes.
feed images
<class 'tensorflow.python.keras.engine.sequential.Sequential'>
<class 'keras_preprocessing.image.directory_iterator.DirectoryIterator'>
<class 'int'>
9
<class 'str'>
deepslice


C:\Users\nicol\.conda\envs\abba-brainglobe-deepslice\lib\site-packages\keras_preprocessing\image\image_data_generator.py:356: UserWarning: This ImageDataGenerator specifies `samplewise_std_normalization`, which overrides setting of `samplewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


9/9 [==============================] - 2s 228ms/step
convert to float
create panda frame
[0.2779703517664408, 0.6805560089691338, 0.853393280762558, 0.9053807864701491, 1.0, 0.7887473773429594, 0.48263543192523795, 0.4263423048835019, 0.19221561675303975]
cor: (132.0, 414.2159824163973, 114.0, 264.0, 432.76144941309417, 228.0)
[-132.          -4.1658549    0.       ]
cor: (132.0, 415.9965739590657, 114.0, 264.0, 432.7614494130941, 228.0)
[   0.          -12.59902055 -114.        ]
cor: (132.0, 420.2971778388468, 114.0, 264.0, 432.76144941309417, 228.0)
[-132.            0.13474898    0.        ]
cor: (132.0, 420.2963177807301, 114.0, 264.0, 432.76144941309406, 228.0)
[   0.          -12.59988061 -114.        ]
cor: (132.0, 420.2701583880634, 114.0, 264.0, 432.76144941309406, 228.0)
[-1.32000000e+02  1.08589585e-01  0.00000000e+00]
cor: (132.0, 420.27015838850105, 114.0, 264.0, 432.7614494130941, 228.0)
[   0.          -12.59988061 -114.        ]
cor: (132.0, 420.270317682598, 114.0, 26

<java object 'java.io.File'>

## Download serial sections examples

Download sections from the zenodo repository: https://zenodo.org/record/6592478#.YpSyc1RByUk (around 1Mb per section)

Files are put in the current repository, under the `images` folder. If files have already been downloaded, the download will be skipped.

In [7]:
import os
from bg_atlasapi import utils
from pathlib import Path
cwd = os.getcwd() # gets current path

utils.check_internet_connection()
base_zenodo_url = 'https://zenodo.org/record/4715656/'

basePath = cwd+'/images/'

def downloadIfNecessary(section_name):
    outputPath = Path(basePath+section_name)
    if not outputPath.exists():
        url = 'https://zenodo.org/record/6592478/files/'+section_name+'?download=1'
        utils.retrieve_over_http(url, outputPath)
    
downloadIfNecessary('S00.tif') 
downloadIfNecessary('S10.tif') 
downloadIfNecessary('S20.tif') 
downloadIfNecessary('S30.tif') 
downloadIfNecessary('S40.tif') 
downloadIfNecessary('S50.tif') 
downloadIfNecessary('S60.tif') 
downloadIfNecessary('S70.tif') 
downloadIfNecessary('S80.tif') 


In [8]:
# Let's get the multipositioner object 
MultiSlicePositioner = jimport('ch.epfl.biop.atlas.aligner.MultiSlicePositioner')

# There's only one multipositioner instance in the object service
# https://javadoc.scijava.org/SciJava/org/scijava/object/ObjectService.html
mp = ij.object().getObjects(MultiSlicePositioner).get(0)


In [9]:
# Let's import the files using Bio-Formats.
# The list of all commands is accessible here:
# https://github.com/BIOP/ijp-imagetoatlas/tree/master/src/main/java/ch/epfl/biop/atlas/aligner/command

ImportImageCommand = jimport('ch.epfl.biop.atlas.aligner.command.ImportImageCommand')

# Here we want to import images: check
# https://github.com/BIOP/ijp-imagetoatlas/blob/master/src/main/java/ch/epfl/biop/atlas/aligner/command/ImportImageCommand.java

file_s00 = File(basePath+'S00.tif')
file_s10 = File(basePath+'S10.tif')
file_s20 = File(basePath+'S20.tif')
file_s30 = File(basePath+'S30.tif')
file_s40 = File(basePath+'S40.tif')
file_s50 = File(basePath+'S50.tif')
file_s60 = File(basePath+'S60.tif')
file_s70 = File(basePath+'S70.tif')
file_s80 = File(basePath+'S80.tif')

FileArray = JArray(File)
files = FileArray(9)

files[0] = file_s00
files[1] = file_s10
files[2] = file_s20
files[3] = file_s30
files[4] = file_s40
files[5] = file_s50
files[6] = file_s60
files[7] = file_s70
files[8] = file_s80

# Any missing input parameter will lead to a popup window asking the missing argument to the user
ij.command().run(ImportImageCommand, True,\
                 "datasetname", JString('dataset'),\
                 "files", files,\
                 "mp", mp,\
                 "split_rgb_channels", False,\
                 "slice_axis_initial", 5.0,\
                 "increment_between_slices", 0.04\
                )



<java object 'java.util.concurrent.FutureTask'>

[java.lang.Enum.toString] TiffDelegateReader initializing C:\Users\chiarutt\Dropbox\BIOP\ABBA-Python\notebooks\images\S00.tif
[java.lang.Enum.toString] Reading IFDs
[java.lang.Enum.toString] Populating metadata
[java.lang.Enum.toString] Checking comment style
[java.lang.Enum.toString] Populating OME metadata
[java.lang.Enum.toString] Attempts to set opener settings for file format Tagged Image File Format; data location = C:\Users\chiarutt\Dropbox\BIOP\ABBA-Python\notebooks\images\S00.tif
[java.lang.Enum.toString] TiffDelegateReader initializing C:\Users\chiarutt\Dropbox\BIOP\ABBA-Python\notebooks\images\S10.tif
[java.lang.Enum.toString] Reading IFDs
[java.lang.Enum.toString] Populating metadata
[java.lang.Enum.toString] Checking comment style
[java.lang.Enum.toString] Populating OME metadata
[java.lang.Enum.toString] Attempts to set opener settings for file format Tagged Image File Format; data location = C:\Users\chiarutt\Dropbox\BIOP\ABBA-Python\notebooks\images\S10.tif
[java.lang.E

[java.lang.Enum.toString] Creating slice S10.tif-ch0...
[java.lang.Enum.toString] Slice S10.tif-ch0 created!
[java.lang.Enum.toString] Multipositioner : Slice added
[java.lang.Enum.toString] Creating slice S20.tif-ch0...
[java.lang.Enum.toString] Slice S20.tif-ch0 created!
[java.lang.Enum.toString] Multipositioner : Slice added
[java.lang.Enum.toString] Creating slice S30.tif-ch0...
[java.lang.Enum.toString] Slice S30.tif-ch0 created!
[java.lang.Enum.toString] Multipositioner : Slice added
[java.lang.Enum.toString] Creating slice S40.tif-ch0...
[java.lang.Enum.toString] Slice S40.tif-ch0 created!
[java.lang.Enum.toString] Multipositioner : Slice added
[java.lang.Enum.toString] Creating slice S50.tif-ch0...
[java.lang.Enum.toString] Slice S50.tif-ch0 created!
[java.lang.Enum.toString] Multipositioner : Slice added
[java.lang.Enum.toString] Creating slice S60.tif-ch0...
[java.lang.Enum.toString] Slice S60.tif-ch0 created!
[java.lang.Enum.toString] Multipositioner : Slice added
[java.lang

In [16]:
RegistrationDeepSliceCommand = jimport('ch.epfl.biop.atlas.aligner.command.RegistrationDeepSliceCommand')

# Any missing input parameter will lead to a popup window asking the missing argument to the user
ij.command().run(RegistrationDeepSliceCommand, True,\
                 "slices_string_channels", JString('0,1'),\
                 "image_name_prefix", JString('Section'),\
                 "mp", mp,\
                 "allow_slicing_angle_change", True,\
                 "allow_change_slicing_position", True,\
                 "maintain_slices_order", True,\
                 "affine_transform", True,\
                 "deepSliceProcessor", run_deep_slice
                )



<java object 'java.util.concurrent.FutureTask'>

[java.lang.Enum.toString] Export of slice S20.tif-ch0 done (3/9)[java.lang.Enum.toString] 
[java.lang.Enum.toString] Export of slice S50.tif-ch0 done (6/9)[java.lang.Enum.toString] 
[java.lang.Enum.toString] Export of slice S10.tif-ch0 done (2/9)[java.lang.Enum.toString] 
[java.lang.Enum.toString] Export of slice S30.tif-ch0 done (4/9)[java.lang.Enum.toString] 
[java.lang.Enum.toString] Export of slice S60.tif-ch0 done (7/9)[java.lang.Enum.toString] 
[java.lang.Enum.toString] Export of slice S40.tif-ch0 done (5/9)[java.lang.Enum.toString] 
[java.lang.Enum.toString] Export of slice S00.tif-ch0 done (1/9)[java.lang.Enum.toString] 
[java.lang.Enum.toString] Export of slice S70.tif-ch0 done (8/9)[java.lang.Enum.toString] 
[java.lang.Enum.toString] Export of slice S80.tif-ch0 done (9/9)[java.lang.Enum.toString] 
[java.lang.Enum.toString] Export as QuickNii Dataset done - Folder : C:\Users\nicol\Dropbox\BIOP\ABBA-Python\notebooks\deepslice\images[java.lang.Enum.toString] 


set one thread
C:\Users\nicol\Dropbox\BIOP\ABBA-Python\notebooks\deepslice
define_image_generator
Found 9 images belonging to 1 classes.
feed images
<class 'tensorflow.python.keras.engine.sequential.Sequential'>
<class 'keras_preprocessing.image.directory_iterator.DirectoryIterator'>
<class 'int'>
9
<class 'str'>
deepslice
9/9 [==============================] - 2s 218ms/step
convert to float
create panda frame
[0.33500660488800243, 0.5276423637536963, 0.746933357524744, 0.8990545703453988, 0.9460193398628156, 0.6751318780244164, 0.41865415438599674, 0.38589252736802954, 0.24366582297184416]
cor: (132.0, 402.4432192862411, 114.0, 264.0, 418.078942072557, 228.0)
[-132.           -4.13208446    0.        ]
cor: (132.0, 400.2082212889388, 114.0, 264.0, 418.07894207255697, 228.0)
[   0.          -13.73863632 -114.        ]
cor: (132.0, 405.27003672623204, 114.0, 264.0, 418.078942072557, 228.0)
[-132.            0.92973097    0.        ]
cor: (132.0, 405.2711068220638, 114.0, 264.0, 418.0789

[java.lang.Enum.toString] Multipositioner : Slicing angle adjusted to Angle X : 0.11713023840379846 has been updated to 0,120
Angle Y : 0.0042174788025084444 has been updated to 0,007

[java.lang.Enum.toString] Cannot append a transformation to a source of class : TransformedSource[java.lang.Enum.toString] 
[java.lang.Enum.toString] You can try 'mutate' or wrap as transformed Source[java.lang.Enum.toString] 
[java.lang.Enum.toString] Cannot append a transformation to a source of class : TransformedSource[java.lang.Enum.toString] 
[java.lang.Enum.toString] You can try 'mutate' or wrap as transformed Source[java.lang.Enum.toString] 
[java.lang.Enum.toString] Cannot append a transformation to a source of class : TransformedSource[java.lang.Enum.toString] 
[java.lang.Enum.toString] You can try 'mutate' or wrap as transformed Source[java.lang.Enum.toString] 
[java.lang.Enum.toString] Cannot append a transformation to a source of class : TransformedSource[java.lang.Enum.toString] 
[java.lang

In [ ]:
mp.selectSlice(mp.getSlices()) # select all slices

In [ ]:
# run_deep_slice.apply('deepslice/')

In [ ]:
mp.getReslicedAtlas().setRotateY(0.05) # Small correction in Y slicing

In [ ]:
mp.deselectSlice(mp.getSlices()) # deselect all

In [ ]:
mp.selectSlice(mp.getSlices().get(2)) # select the last slice

In [ ]:
# Gets the bigdataviewer view. First let's get the class
BdvMultislicePositionerView = jimport('ch.epfl.biop.atlas.aligner.gui.bdv.BdvMultislicePositionerView')

In [ ]:
# view = ij.object().getObjects(BdvMultislicePositionerView).get(0) # Only one BigDataViewer view
# TODO : use fix in newer version to access the view through the object service

In [ ]:
# The slices are always sorted from small z to high z. To keep track of who's who, reference them before moving them
slice00 = mp.getSlices().get(0) 
slice10 = mp.getSlices().get(1)
slice20 = mp.getSlices().get(2)
slice30 = mp.getSlices().get(3) 
slice40 = mp.getSlices().get(4)
slice50 = mp.getSlices().get(5)
slice60 = mp.getSlices().get(6) 
slice70 = mp.getSlices().get(7)
slice80 = mp.getSlices().get(8)

In [ ]:
mp.moveSlice(slice50,9.5)

In [ ]:
mp.moveSlice(slice40,8.2)

In [ ]:
mp.moveSlice(slice30,7.5)

In [ ]:
# Simple actions are accessible through mp.whatever, but most actions are executed on selected slices
# Almost all actions are executed asynchronously

# For a registration : let's select all slices
mp.selectSlice(mp.getSlices()) # select all

In [ ]:
# Let's run an affine registration on the green slice channel and on the reference atlas channel
# elastix needs to be setup, see https://biop.github.io/ijp-imagetoatlas/installation.html
RegistrationElastixAffineCommand = jimport('ch.epfl.biop.atlas.aligner.command.RegistrationElastixAffineCommand')

ij.command().run(RegistrationElastixAffineCommand, True,
                 "mp", mp,\
                 "pixel_size_micrometer", 40,\
                 "show_imageplus_registration_result", False,\
                 "background_offset_value_moving",0,\
                 "atlas_image_channel",0,\
                 "slice_image_channel",1) # second channel, 0-based


In [ ]:
# Let's try spline
RegistrationElastixSplineCommand = jimport('ch.epfl.biop.atlas.aligner.command.RegistrationElastixSplineCommand')

ij.command().run(RegistrationElastixSplineCommand, True,
                 "mp", mp,\
                 "nb_control_points_x", 12,\
                 "pixel_size_micrometer", 20,\
                 "show_imageplus_registration_result", False,\
                 "background_offset_value_moving",0,\
                 "atlas_image_channel",0,\
                 "slice_image_channel",1) # second channel, 0-based


In [ ]:
# Let's wait for all registration to finish
mp.waitForTasks()